In [237]:
import re
import numpy as np
import pandas as pd
import json
import shutil
import gzip
import os
import wget
import tarfile
import time
from time_converter import Time

- We will start with the users,receipts and the brands last
- We will look at the dataset and create structured data model

# User Data Cleaning

In [200]:
user_df = pd.read_csv('csv_files/users.csv', index_col = 0)

In [286]:
users_df = user_df.copy()

In [281]:
users_df.columns

Index(['_id', 'active', 'createdDate', 'lastLogin', 'role', 'signUpSource',
       'state'],
      dtype='object')

- We have a few Dtype objects in our dataset to observe for. While also checking the int64 and float64

In [203]:
users_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 495 entries, 0 to 494
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   _id           495 non-null    object
 1   active        495 non-null    bool  
 2   createdDate   495 non-null    object
 3   lastLogin     433 non-null    object
 4   role          495 non-null    object
 5   signUpSource  447 non-null    object
 6   state         439 non-null    object
dtypes: bool(1), object(6)
memory usage: 27.6+ KB


- Checking first 5 rows we see a few mor data that is in need of cleaning

In [204]:
users_df.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
1,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
2,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
3,{'$oid': '5ff1e1eacfcf6c399c274ae6'},True,{'$date': 1609687530554},{'$date': 1609687530597},consumer,Email,WI
4,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI


In [265]:
id_list = list(users_df[f'lastLogin'].to_dict().values())
date_parsed = id_list[0].replace('}', "").split(':')[1].replace(" ", "")
date_converted = Time(int(date_parsed[:10]), 'posix').to('dt')
format_date = "%m/%d/%Y"
format_time = "%H:%M:%S"
formated_date = date_converted.strftime(format_date)
formated_time = date_converted.strftime(format_time)

In [331]:
def cleanDateLog(id_data, column=None):
    id_list = list(id_data[f'{column}'].to_dict().values())
    times_list = []
    dates_list = []
    for i in range(len(id_list)):
        try: 
            
            date_parsed = id_list[i].replace('}', "").split(':')[1].replace(" ", "")
            date_converted = Time(int(date_parsed[:10]), 'posix').to('dt')
            format_date = "%m/%d/%Y"
            format_time = "%H:%M:%S"
            formated_date = date_converted.strftime(format_date)
            formated_time = date_converted.strftime(format_time)
            times_list.append(formated_time)
            dates_list.append(formated_date)
        except AttributeError:
            
            date_parsed = str(id_list[i])
            dates_list.append(date_parsed)
            
    return dates_list

In [295]:
def cleanIdCpg(id_data, column=None):
#     print(id_data[f'{column}'])
    # create the dictionary into a list to only get the values
    id_list = list(id_data[f'{column}'].to_dict().values())
    cleanedList = []
    for i in range(len(id_list)):
        parsed_name = id_list[i]
        # remove commas colons and split at quotations
        parsed_names = re.findall("'([^']*)'", id_list[i])
        # these were reoccuring strings that i saw
        cleanedList.append(parsed_names[-1])

    return cleanedList

In [332]:
users_df = user_df.copy()
users_df['_id'] = cleanIdCpg(id_data=users_df, column="_id")
users_df['createdDate'] = cleanDateLog(id_data=users_df, column="createdDate")
users_df["lastLogin"] = cleanDateLog(id_data=users_df, column="lastLogin")

In [335]:
users_df

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,01/03/2021,01/03/2021,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,01/03/2021,01/03/2021,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,01/03/2021,01/03/2021,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,01/03/2021,01/03/2021,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,01/03/2021,01/03/2021,consumer,Email,WI
...,...,...,...,...,...,...,...
490,54943462e4b07e684157a532,True,12/19/2014,03/05/2021,fetch-staff,NaN,NaN
491,54943462e4b07e684157a532,True,12/19/2014,03/05/2021,fetch-staff,NaN,NaN
492,54943462e4b07e684157a532,True,12/19/2014,03/05/2021,fetch-staff,NaN,NaN
493,54943462e4b07e684157a532,True,12/19/2014,03/05/2021,fetch-staff,NaN,NaN


# Brand Data Cleaning

In [ ]:
def BrandIdCpg(id_data, column=None):
#     print(id_data[f'{column}'])
    # create the dictionary into a list to only get the values
    id_list = list(id_data[f'{column}'].to_dict().values())
    cleanedList = []
    for i in range(len(id_list)):
        parsed_name = id_list[i]
        # remove commas colons and split at quotations
        parsed_names = re.findall("'([^']*)'", id_list[i])
        # these were reoccuring strings that i saw
        remove_list = ['$ref', 'Cogs','$id', '$oid']
        # remove the extra strings and keep the actual id #
        res = [i for i in parsed_names if i not in remove_list]
        cleanedList.append(parsed_names[-1])

In [196]:
brand_df['_id'] = cleanIdCpg(users_df, column = '_id')

In [198]:
brand_df['cpg'] = cleanIdCpg(users_df, column = 'cpg')